In [ ]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt

RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]


In [ ]:
df = dt.get_posts()

In [ ]:
tdf = pd.read_parquet(os.path.join(RAW_DATA_PATH, "territorytransfer.parquet"))
tdf

In [ ]:
# territory posting fees

mask = (~df['bio']) & (~df['freebie']) & (df['cost']>0) & (df['cost_modifier']==0) & \
       (df['n_uploads']==0) & (~df['id'].isin(globals.perennial_item_ids)) & \
       (df['userId'] != globals.anon_id) & (df['cost']>0) & \
       (df['invoiceActionState'] != 'FAILED')

my_df = df.loc[mask, ['subName', 'created_at', 'cost']].sort_values(
    by=['subName', 'created_at'], ascending=True
).reset_index(drop=True)

# smooth over
for idx in range(1, len(my_df)-1):
    prev_sub = my_df.at[idx-1, 'subName']
    curr_sub = my_df.at[idx, 'subName']
    next_sub = my_df.at[idx+1, 'subName']
    prev_cost = my_df.at[idx-1, 'cost']
    curr_cost = my_df.at[idx, 'cost']
    next_cost = my_df.at[idx+1, 'cost']
    if (prev_sub == curr_sub) and (next_sub == curr_sub) and (prev_cost == next_cost):
        my_df.at[idx, 'cost'] = prev_cost

plot_df = my_df.loc[my_df['subName']=='tech']
plt.step(plot_df['created_at'], plot_df['cost'], where='post')
plt.xticks(rotation=45)
plt.show()


In [ ]:
df.loc[mask & (df['subName']=='bitcoin')].sort_values(by='created_at', ascending=True).to_csv("temp.csv")

In [ ]:
df.loc[ df['userId']==14515].sort_values(by='created_at', ascending=True).to_csv("temp2.csv")

In [ ]:
idx = (df['subName']=='devs') & (df['created_at'].dt.date == pd.to_datetime('2025-04-08').date())
df.loc[idx]

In [ ]:
idx = (df['subName']=='devs') & (df['userId']==616) & (df['created_at'].dt.date == pd.to_datetime('2025-04-08').date())
df.loc[idx]

In [ ]:
idx = (df['userId']==616) & (df['created_at'].dt.date == pd.to_datetime('2025-04-08').date())
df.loc[idx].sort_values(by='created_at', ascending=True)

In [ ]:
itemact_df = pd.read_parquet(os.path.join(RAW_DATA_PATH, "itemact.parquet"))
itemact_df.loc[(itemact_df['itemId']==937934) & (itemact_df['userId']==616)]

In [ ]:
item_df = pd.read_parquet(os.path.join(RAW_DATA_PATH, "item.parquet"))
for col in item_df.columns:
    print(col)
    print(item_df.loc[item_df['id']==886922, col].iloc[0])
    print('')